<a href="https://colab.research.google.com/github/Leehoryeong/DeepLeangingStudy/blob/master/Part8_TensorFlow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
%tensorflow_version 1.x
import tensorflow as tf

In [9]:
# 상수노드 정의
a = tf.constant(1.0,name='a')
b = tf.constant(2.0,name='b')
c = tf.constant([[1.0,2.0],[3.0,4.0]])

print(a)
print(a+b)
print(c)

Tensor("a_1:0", shape=(), dtype=float32)
Tensor("add_4:0", shape=(), dtype=float32)
Tensor("Const_1:0", shape=(2, 2), dtype=float32)


In [10]:
  # 세션 생성, 노드간의 텐서 연산 실행
  # 세션을 통해 노드에 값 할당
sess = tf.Session()
print(sess.run([a,b]))

print(sess.run(c))
print(sess.run([a+b]))
print(sess.run(c+1.0))
sess.close()

[1.0, 2.0]
[[1. 2.]
 [3. 4.]]
[3.0]
[[2. 3.]
 [4. 5.]]


In [11]:
# 플레이스홀더 노드 - 임의의 값 입력 받는 경우
a = tf.placeholder(tf.float32)
b = tf.placeholder(tf.float32)
c = a+b


# 노드간의 연산을 위해 세션 생성
sess = tf.Session()
# c: 실행하고자 하는 연산 feed_dict:실제로 넣을 값 
print(sess.run(c, feed_dict={a:1.0, b:3.0}))
print(sess.run(c, feed_dict={a:[1.0,2.0], b:[3.0,4.0]}))

d = 100 * c
# d: 실행하고자 하는 연산 feed_dict:실제로 넣을 값 

print(sess.run(d, feed_dict={a:1.0, b:3.0}))
print(sess.run(d, feed_dict={a:[1.0,2.0], b:[3.0,4.0]}))

# 세션 close
sess.close()

# 플레이스홀더 노드는 입력, 정답데이터를 넣어 주기 위해사용 

4.0
[4. 6.]
400.0
[400. 600.]


In [12]:
# 변수 노드 - 계속해서 업데이트 되는 변수
W1 = tf.Variable(tf.random_normal([1])) #가우시안 분포의 임의의값 초기 변수로 사용
b1 = tf.Variable(tf.random_normal([1]))

W2 = tf.Variable(tf.random_normal([1,2]))
b2 = tf.Variable(tf.random_normal([1,2]))

# 세션 생성
sess = tf.Session()
# <매우중요>변수노드 값 초기화, 변수 노드를 정의했다면 위해 반드시 실행
sess.run(tf.global_variables_initializer())

for step in range(3):
  W1 = W1 - step
  b1 = b1 - step
  W2 = W2 - step
  b2 = b2 - step  

  print('step = ', step, 'W1 = ',sess.run(W1),'b1 = ',sess.run(b1))
  print('step = ', step, 'W2 = ',sess.run(W2),'b2 = ',sess.run(b2))

step =  0 W1 =  [0.84073114] b1 =  [-1.056081]
step =  0 W2 =  [[0.92338514 0.04250633]] b2 =  [[-0.82944494  0.4055577 ]]
step =  1 W1 =  [-0.15926886] b1 =  [-2.056081]
step =  1 W2 =  [[-0.07661486 -0.95749366]] b2 =  [[-1.8294449 -0.5944423]]
step =  2 W1 =  [-2.1592689] b1 =  [-4.056081]
step =  2 W2 =  [[-2.0766149 -2.9574938]] b2 =  [[-3.829445  -2.5944424]]


# linear Reg 구현


In [13]:
import numpy as np

In [14]:
# 1. 데이터 분리
loaded_data = np.loadtxt('/content/data-01.csv',delimiter=',')
x_data = loaded_data[:,0:-1]
t_data = loaded_data[:,[-1]]
print(x_data.shape, t_data.shape)

(25, 3) (25, 1)


In [15]:
# 2. 데이터 입력
W = tf.Variable(tf.random_normal([3,1]))
b = tf.Variable(tf.random_normal([1]))

X = tf.placeholder(tf.float32, [None,3]) # None을 사용하면 몇개의 행이와도 열만 맞으면 사용 가능
T = tf.placeholder(tf.float32, [None,1]) 


In [16]:
# 3. 선형 회귀
# 4. 손실 함수
y = tf.matmul(X,W) + b
loss = tf.reduce_mean(tf.square(y-T)) # MSE 손실함수 정의 

In [17]:
# 5. W,b 최적화(경사하강법)
learning_rate = 1e-5
optimizer = tf.train.GradientDescentOptimizer(learning_rate) # 경사하강법 알고리즘 적용되는 optimizer
# 최적화 알고리즘은 다수 존재
train = optimizer.minimize(loss) #손실함수 최소화

In [18]:
with tf.Session() as sess: # with 사용하면 session.close 해주지 않아도 됨
  sess.run(tf.global_variables_initializer()) # 변수 노드 tf.Variable 초기화
  for step in range(8001):
    loss_val, y_val, _ = sess.run([loss,y,train], feed_dict={X:x_data, T:t_data})
    if step % 400 ==0:
      print('stpe= ',step,'loss_val= ',loss_val)
  print('\nPrediction is', sess.run(y, feed_dict={X:[[100,98,81]]}))

stpe=  0 loss_val=  16202.138
stpe=  400 loss_val=  60.859207
stpe=  800 loss_val=  44.53538
stpe=  1200 loss_val=  33.087692
stpe=  1600 loss_val=  25.05846
stpe=  2000 loss_val=  19.425756
stpe=  2400 loss_val=  15.473666
stpe=  2800 loss_val=  12.7001
stpe=  3200 loss_val=  10.753091
stpe=  3600 loss_val=  9.385851
stpe=  4000 loss_val=  8.425397
stpe=  4400 loss_val=  7.750395
stpe=  4800 loss_val=  7.275797
stpe=  5200 loss_val=  6.941881
stpe=  5600 loss_val=  6.706813
stpe=  6000 loss_val=  6.541179
stpe=  6400 loss_val=  6.4243536
stpe=  6800 loss_val=  6.341858
stpe=  7200 loss_val=  6.2835364
stpe=  7600 loss_val=  6.2422314
stpe=  8000 loss_val=  6.2129297

Prediction is [[179.33842]]


# Logistic Reg



In [19]:
# 1. 데이터 분리
loaded_data = np.loadtxt('/content/diabetes.csv',delimiter=',')
x_data = loaded_data[:,0:-1]
t_data = loaded_data[:,[-1]]
print(x_data.shape, t_data.shape)

(759, 8) (759, 1)


In [20]:
# 2. 데이터 입력
W = tf.Variable(tf.random_normal([8,1]))
b = tf.Variable(tf.random_normal([1]))

X = tf.placeholder(tf.float32, [None,8]) # None을 사용하면 몇개의 행이와도 열만 맞으면 사용 가능
T = tf.placeholder(tf.float32, [None,1]) 


In [21]:
# 3. 선형 회귀
# 4. 손실 함수
Z = tf.matmul(X,W) + b
y = tf.sigmoid(Z)

loss =  -tf.reduce_mean(T*tf.log(y) + (1-T)*tf.log(1-y))

In [22]:
learning_rate = 0.01
optimizer = tf.train.GradientDescentOptimizer(learning_rate) # 오차 역전파를 이용해 W,b 최적화
train = optimizer.minimize(loss)

In [23]:
predicted = tf.cast(y>0.5,dtype=tf.float32) # sigmoid값이 0.5보다 크면 True
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted,T), dtype=tf.float32))
# equal -> predicted = y => True else False
# cast -> True, False를 1,0 으로 변경
# reduce_mean -> 평균계산

In [24]:
with tf.Session() as sess:
  sess.run(tf.global_variables_initializer()) #변수 노드 초기화
  for step in range(20001):
    loss_val, _ = sess.run([loss,train],feed_dict={X:x_data,T:t_data})

    if step % 500 ==0:
      print('step = ',step, 'loss_val = ',loss_val)

  y_val, predicted_val, accuracy_val = sess.run([y,predicted,accuracy], feed_dict={X:x_data, T:t_data})
  print('\ny_val.shape = ',y_val.shape,', predicted_val = ',predicted_val.shape)
  print('\nAccuracy = ',accuracy_val)

step =  0 loss_val =  0.7174718
step =  500 loss_val =  0.66818476
step =  1000 loss_val =  0.62913656
step =  1500 loss_val =  0.59839255
step =  2000 loss_val =  0.57432
step =  2500 loss_val =  0.55548316
step =  3000 loss_val =  0.54069406
step =  3500 loss_val =  0.52901196
step =  4000 loss_val =  0.51971394
step =  4500 loss_val =  0.5122515
step =  5000 loss_val =  0.50621164
step =  5500 loss_val =  0.5012824
step =  6000 loss_val =  0.4972275
step =  6500 loss_val =  0.49386632
step =  7000 loss_val =  0.49106023
step =  7500 loss_val =  0.48870182
step =  8000 loss_val =  0.48670688
step =  8500 loss_val =  0.48500934
step =  9000 loss_val =  0.48355678
step =  9500 loss_val =  0.48230726
step =  10000 loss_val =  0.4812269
step =  10500 loss_val =  0.48028842
step =  11000 loss_val =  0.47946963
step =  11500 loss_val =  0.47875217
step =  12000 loss_val =  0.47812104
step =  12500 loss_val =  0.47756362
step =  13000 loss_val =  0.4770696
step =  13500 loss_val =  0.476630

### 텐서플로우는 은닉층에서는 sigmoid 대신 relu 함수, 출력층에서는 sortmax 함수 사용



In [26]:
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np

In [27]:
mnist = input_data.read_data_sets('MNIST_data/',one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py fr

In [32]:
print('\n',mnist.train.num_examples,mnist.test.num_examples,mnist.validation.num_examples)
print('train image shape',np.shape(mnist.train.images))
print('train label shape',np.shape(mnist.train.labels))
print(np.shape(mnist.test.images))
print(np.shape(mnist.test.labels))


 55000 10000 5000
train image shape (55000, 784)
train label shape (55000, 10)
(10000, 784)
(10000, 10)


In [33]:
# 하이퍼파라미터는 컴퓨터 성능에 따라 변경 가능
learning_rate = 0.1   # 학습율
epochs = 100          # 반복횟수
batch_size = 100      # 한번에 입력으로 주어지는 MNIST 개수

input_nodes = 784     # 입력노드 개수
hidden_nodes = 100    # 은닉노드 개수
output_nodes = 10     # 출력노드 개수

In [35]:
X = tf.placeholder(tf.float32, [None, input_nodes])
T = tf.placeholder(tf.float32, [None, output_nodes]) #Target

W2 = tf.Variable(tf.random_normal([input_nodes, hidden_nodes]))
b2 = tf.Variable(tf.random_normal([hidden_nodes]))

W3 = tf.Variable(tf.random_normal([hidden_nodes, output_nodes]))
b3 = tf.Variable(tf.random_normal([output_nodes]))

In [38]:
Z2 = tf.matmul(X,W2) + b2
A2 = tf.nn.relu(Z2)

Z3 = logits = tf.matmul(A2,W3) + b3
y = A3 = tf.nn.softmax(Z3)

In [39]:
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=Z3,labels=T))

optimizer = tf.train.GradientDescentOptimizer(learning_rate)
train = optimizer.minimize(loss)

In [41]:
predicted_val = tf.equal(tf.argmax(A3,1),tf.argmax(T,1))
accuracy = tf.reduce_mean(tf.cast(predicted_val,dtype=tf.float32))

In [42]:

with  tf.Session()  as sess:
    
    sess.run(tf.global_variables_initializer())  # 변수 노드(tf.Variable) 초기화
    
    for i in range(epochs):    # 100 번 반복수행
        
        total_batch = int(mnist.train.num_examples / batch_size)  # 55,000 / 100

        for step in range(total_batch):
            
            batch_x_data, batch_t_data = mnist.train.next_batch(batch_size)
      
            loss_val, _ = sess.run([loss, train], feed_dict={X: batch_x_data, T: batch_t_data})    
        
            if step % 100 == 0:
                print("step = ", step, ", loss_val = ", loss_val)             
    
    # Accuracy 확인
    test_x_data = mnist.test.images    # 10000 X 784
    test_t_data = mnist.test.labels    # 10000 X 10
    
    accuracy_val = sess.run(accuracy, feed_dict={X: test_x_data, T: test_t_data})
    
    print("\nAccuracy = ", accuracy_val)

step =  0 , loss_val =  103.23072
step =  100 , loss_val =  3.9515078
step =  200 , loss_val =  3.293043
step =  300 , loss_val =  1.3089633
step =  400 , loss_val =  1.6956375
step =  500 , loss_val =  1.5029268
step =  0 , loss_val =  2.1039681
step =  100 , loss_val =  1.1662147
step =  200 , loss_val =  0.6804744
step =  300 , loss_val =  0.69596696
step =  400 , loss_val =  0.56577
step =  500 , loss_val =  0.81567377
step =  0 , loss_val =  0.80993456
step =  100 , loss_val =  0.40694743
step =  200 , loss_val =  1.1690364
step =  300 , loss_val =  0.63719755
step =  400 , loss_val =  0.98029715
step =  500 , loss_val =  0.56272936
step =  0 , loss_val =  0.31872058
step =  100 , loss_val =  0.52275705
step =  200 , loss_val =  0.31038764
step =  300 , loss_val =  0.20409761
step =  400 , loss_val =  0.43583575
step =  500 , loss_val =  0.43886262
step =  0 , loss_val =  0.5324902
step =  100 , loss_val =  0.68017626
step =  200 , loss_val =  0.5487034
step =  300 , loss_val =  0